In [1]:
from tensorflow.contrib.keras.python.keras import applications, layers
from tensorflow.contrib.keras.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras.python.keras import optimizers
from tensorflow.contrib.keras.python.keras.models import Sequential, Model
from tensorflow.contrib.keras.python.keras.layers import Dropout, Flatten, Dense, Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping
from tensorflow.contrib.keras.python.keras.optimizers import Adam

# from tensorflow.contrib.keras.python.keras.applications.resnet50 import ResNet50
from tensorflow.contrib.keras.python.keras.applications.imagenet_utils import _obtain_input_shape
from tensorflow.contrib.keras.python.keras import backend as K

from residual_funcs import *

# Create the base model ResNet50

In [2]:
# Determine proper input shape
input_shape = _obtain_input_shape(None,
                                default_size=224,
                                min_size=197,
                                data_format=K.image_data_format(),
                                include_top=False)

img_input = Input(shape=input_shape)

In [3]:
bn_axis = 3

x = ZeroPadding2D((3, 3))(img_input)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

x = AveragePooling2D((7, 7), name='avg_pool')(x)

In [4]:
inputs = img_input

# Create model.
base_model = Model(inputs, x, name='resnet50')

In [5]:
# Load trained model
base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

# FC layer

In [6]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add 2 fully-connected layers
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(5, activation='softmax')(x)

In [7]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)
opt = Adam(lr=0.0001, decay=10e-6)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
     "./data/train",
    target_size=(224, 244),
    batch_size=128,
    class_mode='binary')

val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow_from_directory(
     "./data/val",
    target_size=(224, 244),
    batch_size=95,
    class_mode='binary')

model.fit_generator(generator=train_generator, steps_per_epoch=20,
                    epochs=100, verbose=1, callbacks=[early_stopping],
                    validation_data=val_generator, validation_steps=10)

Found 2720 images belonging to 5 classes.
Found 950 images belonging to 5 classes.
Epoch 1/100
20/20 [==============================] - 32s - loss: 0.8329 - acc: 0.6902 - val_loss: 0.7020 - val_acc: 0.7263
Epoch 2/100
20/20 [==============================] - 28s - loss: 0.3563 - acc: 0.8813 - val_loss: 0.5293 - val_acc: 0.8063
Epoch 3/100
20/20 [==============================] - 28s - loss: 0.2664 - acc: 0.9077 - val_loss: 0.4990 - val_acc: 0.8221
Epoch 4/100
20/20 [==============================] - 28s - loss: 0.1972 - acc: 0.9371 - val_loss: 0.3957 - val_acc: 0.8568
Epoch 5/100
20/20 [==============================] - 28s - loss: 0.1227 - acc: 0.9660 - val_loss: 0.3471 - val_acc: 0.8663
Epoch 6/100
20/20 [==============================] - 28s - loss: 0.1182 - acc: 0.9633 - val_loss: 0.3566 - val_acc: 0.8758
Epoch 7/100
20/20 [==============================] - 28s - loss: 0.0961 - acc: 0.9755 - val_loss: 0.2959 - val_acc: 0.8937
Epoch 8/100
20/20 [==============================] - 28s